# Modelo A

por: Mariana Lozano Roncancio

## Seccion 1: Setup del cuaderno

In [1]:
import pandas as pd
from ydata_profiling import ProfileReport
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import matplotlib

# import word tokenize
from nltk.tokenize import word_tokenize
import nltk
import numpy as np
from sklearn.feature_extraction.text import (
    TfidfVectorizer,
    CountVectorizer,
    HashingVectorizer,
)

nltk.download("punkt_tab")
import spacy
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import joblib
from collections import Counter
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

# Load the large spanish model
nlp = spacy.load("es_core_news_lg")
# nlp = spacy.load('es_dep_news_trf')
%matplotlib inline

c:\Users\MarianaLozano\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\MarianaLozano\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


## Seccion 2 - entendimiento de los datos

In [5]:
df_original = data = pd.read_csv('./datos/fake_news_spanish.csv', sep=';', encoding='utf-8', on_bad_lines='warn')
#df_test_original = pd.read_excel("./datos/fake_news_test.csv")


In [4]:
df_original

,ID,Label,Titulo,Descripcion,Fecha
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018
...,...,...,...,...,...
57058,ID,1,El Defensor del Pueblo reclama a la Comunidad ...,El gobierno regional han indicado que la atenc...,08/06/2021
57059,ID,0,El EQUO plantea ceder la presidencia de la Com...,Si la higiene democrática nos lleva a esa exig...,08/09/2020
57060,ID,1,Alberto Garzón: 'Que los Borbones son unos lad...,El coordinador federal de IU asegura que la mo...,12/07/2018
57061,ID,1,Vox exige entrar en el Gobierno de Castilla y ...,Santiago Abascal: Vox tiene el derecho y el de...,13/02/2022


## Calidad de los datos

Dimensiones de calidad de los datos:

- Consistencia
- Validez
- Completitud
- Unicidad

## Complejitud
Bajo una inspección sencilla, parece no haber problemas de completitud en los datos

In [7]:
df_original["Label"].value_counts()

Label
1    33158
0    23905
Name: count, dtype: int64

In [8]:
# strip all values in the column
df_original['Descripcion'] = df_original['Descripcion'].str.strip()
# count all values in the column where the value is an empty string

df_original['Descripcion'].isnull().sum()

np.int64(0)

## Validez

Se identifican los siguietnes problemas de validez:

- Hay un problema de codificación de los textos en el dataset, pues muchos caracteres que corresponden a vocales con tilde han sido reemplazados por caracteres no válidos.

In [12]:
def fix_encoding(text):
    replacements = {
        'Ã¡': 'á',
        'Ã©': 'é',
        'Ã­': 'í',
        'Ã³': 'ó',
        'Ãº': 'ú',
        'Ã±': 'ñ'
    }
    
    for wrong, correct in replacements.items():
        text = text.replace(wrong, correct)
    
    return text

df_original['Descripcion'] = df_original['Descripcion'].apply(fix_encoding)


In [10]:
df_original.head(5)

,ID,Label,Titulo,Descripcion,Fecha
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018


## Consistencia
Algunas de las entradas de texto del dataset están rodeadas por comillas, aunque no todas. Es posible que esto no introduzca problemas en el dataset.


## Unicidad

En primera instancia, el dataset no tiene columnas que deban ser únicas siguiento algúna restricción de negocio. Bajo una inspección sencilla, parece haber filas totalmente duplicadas.

In [13]:
df_original.duplicated().sum()

np.int64(445)

In [14]:
df_original.drop_duplicates(inplace=True)

## Perfilamiento de los datos

Se aprovecha el paquete ydata-profiling para hacer un perfilamiento detallado del dataset. A partir de este reporte se destacan las siguientes conclusiones:

In [15]:
profile = ProfileReport(df_original, title="Profiling Report")
profile.to_file("./data_profiling.html")
profile

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 111.76it/s]


ModuleNotFoundError: No module named 'ipywidgets'